In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql import functions as F

spark = SparkSession.builder \
.getOrCreate()

web_server_logs_df = spark.read \
.option('header', True) \
.csv('/content/web_server_logs.csv')

web_server_logs_df = (web_server_logs_df
.withColumn('timestamp', F.to_timestamp(col('timestamp'),
            "yyyy-MM-dd'T'HH:mm:ss.SSSSSS"))
.withColumn('date', F.to_date(col('timestamp')))
.withColumn('response_size', col('response_size').cast('int'))
.withColumn('response_code', col('response_code').cast('int'))
)

web_server_logs_df.createOrReplaceTempView('web_server_logs')

query = """
        SELECT ip, COUNT(*) as request_count
        FROM web_server_logs
        GROUP BY ip
        ORDER BY request_count DESC
        LIMIT 10
        """


query1 = """
         SELECT method, COUNT(*) as method_count
         FROM web_server_logs
         GROUP BY method
         ORDER BY method_count DESC
         """

query2 = """
         SELECT date, SUM(response_size) as total_response_size
         FROM web_server_logs
         WHERE response_code = 404
         GROUP BY date
         ORDER BY date DESC
         """




cnt_404 = web_server_logs_df.filter(F.col('response_code') == 404).count()



print('Top 10 active IP addresses:')
spark.sql(query).show(truncate=False)
print('Request countby HTTP method:')
spark.sql(query1).show(truncate=False)
print(f'Number of 404 response codes: {cnt_404}')
print('Total response size by day:')
spark.sql(query2).show(truncate=False)